<a href="https://colab.research.google.com/github/AlexeyUdod/aml/blob/master/dll_5_hw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
Задание 1.
Обучите нейронную сеть решать шифр цезаря.
Что надо сделать:
1.Написать алгоритм шифра цезаря для генерации выборки (сдвиг на К каждой буквы. Например, при сдвиге на 2 буква “А” переходит в букву “В” и тп)
2.Сделать нейронную сеть
3.Обучить ее (вход - зашифрованная фраза, выход - дешифрованная фраза)
4.Проверить качество

Задание 2.
Выполнить практическую работу из лекционного ноутбука.
а) построить RNN-ячейку на основе полносвязных слоев
б) применить построенную ячейку для генерации текста с выражениями героев сериала “Симпсоны”

In [6]:
import torch as tr
import re
from itertools import product
from functools import reduce
import pandas as pd
import copy
import matplotlib.pyplot as plt
import seaborn as sns
t = tr.tensor

In [2]:
dev = tr.device('cuda' if tr.cuda.is_available() else 'cpu')
if dev.type == 'cuda':
    print(tr.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(tr.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(tr.cuda.memory_reserved(0)/1024**3,1), 'GB')
else:
    print('CPU')

Tesla P100-PCIE-16GB
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [3]:
def i2l(text, dic): 
    return [''.join([dic[letter.item()] for letter in sentense]) for sentense in text]

def l2i(text, dic): 
    adic = {v:k  for k, v in dic.items()}
    return tr.tensor([[adic[letter] for letter in sentense] for sentense in text])

def text2cesar(text, shift, dic):
    n_classes = len(dic)
    return ''.join(i2l((l2i(text,dic) + shift) % n_classes, dic))

In [4]:
class rnn1(tr.nn.Module):
    def __init__(self, emb_size, hidden_size, n_classes, num_layers):
        super().__init__()
        self.emb = tr.nn.Embedding(n_classes, emb_size)
        self.rnn = tr.nn.RNN(emb_size, hidden_size, num_layers, batch_first=True)
        self.lin = tr.nn.Linear(hidden_size, n_classes)
        self.hidden_size = hidden_size
    def forward(self, input):
        r = self.emb(input)
        r = self.rnn(r)[0]
        r = self.lin(r).squeeze()
        return r

In [8]:
ab = 'abcdefghijklmnopqrstuvwxyz ,.'
dic = dict(zip(range(len(ab)), ab))

n_classes = len(dic)
cesar_shift = 4
seq_len = 1000
batch_size = 1
n_iters = 1000

model = rnn1(30, 100, len(dic), 1).to(dev)
criterion = tr.nn.CrossEntropyLoss()
optimizer = tr.optim.Adam(model.parameters())
loss_best = 10**10
for iter in range(n_iters):
    text = tr.randint(0, n_classes, (batch_size, seq_len)).to(dev)
    cesar = (text + cesar_shift) % n_classes
    # optimizer.zero_grad()
    pred = model(cesar)
    loss = criterion(pred, text.flatten())#.to(dev)
    if loss < loss_best: 
        model_best = copy.copy(model)
        loss_best = loss
    if iter % (n_iters / 10) == 0: 
        print(f'iter {iter} loss = {loss} ')
    loss.backward()
    optimizer.step()

iter 0 loss = 3.364370346069336 
iter 100 loss = 0.046140220016241074 
iter 200 loss = 0.002146678976714611 
iter 300 loss = 0.0004706206964328885 
iter 400 loss = 0.0005836443742737174 
iter 500 loss = 0.00025856198044493794 
iter 600 loss = 7.28902014088817e-05 
iter 700 loss = 0.001037787296809256 
iter 800 loss = 0.03093920461833477 
iter 900 loss = 0.018807673826813698 


In [10]:
loss_best

tensor(1.1399e-05, device='cuda:0', grad_fn=<NllLossBackward>)

In [11]:
text = '''Las Vegas Last April, John took a trip to Las Vegas, Nevada. 
Las Vegas is a popular destination in the western portion of the United States. 
The town is most popular for its casinos, hotels, and exciting nightlife. '''

text = ''.join(re.findall(f'[a-z,\. ]', text.lower()))[:70]
cesar = text2cesar(text, 4, dic)
res = i2l(model(l2i(cesar, dic).to(dev)).argmax(dim=1).unsqueeze(0), dic)

print(f'Исходный текст:                 {text}\nЗашифрованный текст:            {cesar}\nРасшифрованных моделью текст:   {res[0]}')

Исходный текст:                 las vegas last april, john took a trip to las vegas, nevada. las vegas
Зашифрованный текст:            pewbzikewbpewxbetvmpcbnslrbxssobebxvmtbxsbpewbzikewcbrizehedbpewbzikew
Расшифрованных моделью текст:   las vegas last april, john took a trip to las vegas, nevada. las vegas


In [ ]:
tr.autograd.set_detect_anomaly(True)
n_iter = 1000
shift = 4
seq_len = 10
batch_size = 50
emb_size = 3
hidden_size = 20

w_emb = tr.randn(n_classes, emb_size, requires_grad=True).to(dev)
w_x = tr.randn(emb_size, hidden_size, requires_grad=True).to(dev)
w_h = tr.randn(hidden_size, hidden_size, requires_grad=True).to(dev)
w_o = tr.randn(hidden_size, n_classes, requires_grad=True).to(dev)
bias_h = tr.randn(1, requires_grad=True).to(dev)
bias_o = tr.randn(1, requires_grad=True).to(dev)

h_n = tr.zeros(batch_size, seq_len + 1, hidden_size).to(dev)
h_0 = tr.randn(batch_size, hidden_size).to(dev)
h_n[:,0,:] = h_0
criterion = tr.nn.CrossEntropyLoss()
lr = 0.000000001
loss_hist = []
loss_best = tr.tensor([10]).to(dev)**10
for _ in range(n_iter):
    text = tr.randint(0, n_classes, (batch_size, seq_len)).to(dev)
    cesar = (text + shift) % n_classes
    text_ohe = tr.nn.functional.one_hot(text, num_classes=n_classes).float()
    input = text_ohe @ w_emb
    
    for i0 in range(batch_size):
        for i1 in range(seq_len):
            h_n[i0, i1 + 1, :] = input[i0, i1, :] @ w_x + h_n[i0, i1, :].clone() @ w_h + bias_h
    # h_n = h_n[:,1:,:]
    output = h_n[:,1:,:] @ w_o + bias_o
    loss = criterion(output.view(-1, n_classes), text.flatten())
    loss_hist.append(loss)
    if loss < loss_best:
        model_best = dict(zip(['w_x', 'w_h', 'w_o', 'w_emb', 'bias_h', 'bias_o'], copy.copy([w_x, w_h, w_o, w_emb, bias_h, bias_o])))
        loss_best = loss
    loss.backward(retain_graph=True) 
    if _ % 10 == 0:
        print(f'iter = {_}, loss = {loss.item()}, loss_best = {loss_best.item()}')
    w_x = (w_x.detach() - w_x.grad * lr).requires_grad_(True)
    w_h = (w_h.detach() - w_h.grad * lr).requires_grad_(True)
    w_o = (w_o.detach() - w_o.grad * lr).requires_grad_(True)
    w_emb = (w_emb.detach() - w_emb.grad * lr).requires_grad_(True)
    bias_h = (bias_h.detach() - bias_h.grad * lr).requires_grad_(True)
    bias_o = (bias_o.detach() - bias_o.grad * lr).requires_grad_(True)

iter = 0, loss = 3272192.0, loss_best = 3272192.0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:42: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.


TypeError: ignored

In [ ]:
text = '''Las Vegas Last April, John took a trip to Las Vegas, Nevada. 
Las Vegas is a popular destination in the western portion of the United States. 
The town is most popular for its casinos, hotels, and exciting nightlife. '''

text = [''.join(re.findall(f'[a-z,\. ]', text.lower()))[:70]]
text_ohe = tr.nn.functional.one_hot(l2i(text, dic), num_classes=n_classes).float()
input = text_ohe @ w_emb
# cesar = (text + shift) % n_classes
batch_size = len(text)
seq_len = len(text[0])
# emb_size = 3
hidden_size = 20

h_n = tr.zeros(batch_size, seq_len + 1, hidden_size)
h_0 = tr.randn(batch_size, hidden_size)
h_n[:,0,:] = h_0
for i0 in range(batch_size):
    for i1 in range(seq_len):
        h_n[i0, i1 + 1, :] = input[i0, i1, :] @ model_best['w_x'] + h_n[i0, i1, :].clone() @ model_best['w_h'] + model_best['bias_h']
output = h_n[:,1:,:] @ model_best['w_o'] + model_best['bias_o']


cesar = text2cesar(text, 4, dic)
res = i2l(output.argmax(dim=-1),dic)

print(f'Исходный текст:                 {text}\nЗашифрованный текст:            {cesar}\nРасшифрованных моделью текст:   {res[0]}')

Исходный текст:                 ['las vegas last april, john took a trip to las vegas, nevada. las vegas']
Зашифрованный текст:            pewbzikewbpewxbetvmpcbnslrbxssobebxvmtbxsbpewbzikewcbrizehedbpewbzikew
Расшифрованных моделью текст:   idiltohn didicqcoqrdmdxcntopidicqcnicqrdxcxobiouobio,idicxrdxoxodiouio


In [12]:
import platform
if platform.system() == 'Linux':
    path = '/content/drive/My Drive/Colab Notebooks/dll/'
    from google.colab import drive
    drive.mount('/content/drive')
else:
    path = ""#"d:\drive\Colab Notebooks\dll\hymenoptera_data\"""

Mounted at /content/drive


In [173]:
def text2tensor(text_in_list, dic, max_letters):
    # text1 = ['like what ','']
    x1 = tr.zeros(len(text_in_list), max_letters).long().to(dev)
    for i0 in range(len(text_in_list)):
        for i1 in range(max_letters):
            if i1 < len(text_in_list[i0]): x1[i0, i1] = dic[text_in_list[i0][i1]]
            else: break
    return x1

def tensor2text(tensor_text, dic):
    adic = {v:k for k, v in dic.items()}
    return [''.join([adic[l.item()] for l in ph]).replace('none', ' ') for ph in tensor_text]

def generate_text(text, dic, model, max_letters):
    x1 = text2tensor([text, ''], dic, max_letters)
    x2 = x1.clone()
    for i in range(len(text), max_letters):
        pred = x2
        pred = model(x2).argmax(dim=-1)
        x2[0,i] = pred[0,i-1]
    return list(zip(tensor2text(x1, dic), tensor2text(x2, dic)))[0]

In [68]:
class rnn2(tr.nn.Module):
    def __init__(self, emb_size, hidden_size, n_classes, num_layers, lin_hidden_size):
        super().__init__()
        self.emb = tr.nn.Embedding(n_classes, emb_size)
        self.rnn = tr.nn.RNN(emb_size, hidden_size, num_layers, batch_first=True)
        self.lin1 = tr.nn.Linear(hidden_size, lin_hidden_size)
        self.relu = tr.nn.ReLU6()
        self.lin2 = tr.nn.Linear(lin_hidden_size, n_classes)
        # self.hidden_size = hidden_size
    def forward(self, input):
        r = self.emb(input)
        r = self.rnn(r)[0]
        r = self.lin1(r)
        r = self.relu(r)
        r = self.lin2(r).squeeze()
        return r

In [206]:
data.normalized_text

0                                   maggie look whats that
1                                          lee-mur lee-mur
2                                          zee-boo zee-boo
3        im trying to teach maggie that nature doesnt e...
4        its like an ox only it has a hump and a dewlap...
                               ...                        
11634    too bad we didnt come dressed as popular carto...
11635    yeah mom guess what for a dollar a man sold me...
11636                                   hows it going bart
11637    maybe you need to play on their sympathies mor...
11638                          ah ha now you look pathetic
Name: normalized_text, Length: 11639, dtype: object

In [162]:
ab = ['none'] + list('abcdefghijklmnopqrstuvwxyz ')
dic = dict(zip(ab, range(len(ab))))

data = pd.read_csv(path + 'data.csv')
text = [''.join(re.findall(f'[a-z ]', t)) for t in data.normalized_text.to_list() if type(t) is str]
max_letters = 100
X = text2tensor(text, dic, max_letters)

In [199]:
n_classes = len(dic)
adic = {v:k for k, v in dic.items()}
batch_size = 100
n_iters = 10

model = rnn2(n_classes, 600, n_classes, 4, 600).to(dev)
criterion = tr.nn.CrossEntropyLoss()
optimizer = tr.optim.AdamW(model.parameters())
loss_best = 10**10
for iter in range(n_iters):
    for i in range(X.shape[0] // batch_size):
        x = X[i:i + batch_size,  :-1].to(dev)
        y = X[i:i + batch_size, 1:  ].to(dev)
        optimizer.zero_grad()
        pred = model(x)
        loss = criterion(pred.reshape(-1, len(dic)), y.flatten()).to(dev)
        if loss < loss_best: 
            model_best = copy.copy(model)
            loss_best = loss
        loss.backward()
        optimizer.step()
        if i % 200 == 0: 
            text1 = [''.join([adic[l.item()] for l in ph]).replace('none', ' ') for ph in x]
            pred1 = [''.join([adic[l.item()] for l in ph]).replace('none', ' ') for ph in model(x).argmax(dim=-1)]
            print(f'iter {iter}, {i*batch_size} of {X.shape[0]} batch, loss = {loss}, loss_best = {loss_best}, text/pred = {text1[0][1:11]}/{pred1[0][0:10]}')

iter 0, 0 of 10891 batch, loss = 3.3325107097625732, loss_best = 3.3325107097625732, text/pred = aggie look/          
iter 1, 0 of 10891 batch, loss = 1.1262502670288086, loss_best = 0.9618717432022095, text/pred = aggie look/hreintaint
iter 2, 0 of 10891 batch, loss = 0.9592496156692505, loss_best = 0.5594141483306885, text/pred = aggie look/oyeie sark
iter 3, 0 of 10891 batch, loss = 0.7352344393730164, loss_best = 0.13298413157463074, text/pred = aggie look/oykie mumk
iter 4, 0 of 10891 batch, loss = 0.289286345243454, loss_best = 0.03496379032731056, text/pred = aggie look/oygie mumk
iter 5, 0 of 10891 batch, loss = 0.14930783212184906, loss_best = 0.02642136439681053, text/pred = aggie look/oygie mumk
iter 6, 0 of 10891 batch, loss = 0.10438606142997742, loss_best = 0.0244036465883255, text/pred = aggie look/oygie mook
iter 7, 0 of 10891 batch, loss = 0.08885058760643005, loss_best = 0.024079686030745506, text/pred = aggie look/oygie nook
iter 8, 0 of 10891 batch, loss = 0.080138

In [205]:
text = 'my name is '
generate_text(text, dic, model_best, 100)

('my name is                                                                                          ',
 'my name is lisa simpson and i have a problem                                                        ')